---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [24]:

import networkx as nx
import pandas as pd
import numpy as np
import pickle


---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [25]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [26]:
def graph_identification():
    '''Get the information about the graph and then use the names and the values to determine the algoritthm that was used to 
    develop the graph'''
    names=[]
    for graph in P1_Graphs:
        parts= nx.info(graph).split('\n')
        names.append(parts[0].split(' ')[1])
    used_algo= []
    p_watts=[]
    for name in names:
        if 'barabasi' in name:
            used_algo.append('PA')
        elif 'watts' in name and float(name[name.rfind(',')+1:name.rfind(')')])<0.1:
            used_algo.append('SW_L')
        else:
            used_algo.append('SW_H')
            
            
    return used_algo



---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [27]:
G = nx.read_gpickle('email_prediction.txt')


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [28]:
def salary_predictions():
    from sklearn.linear_model import LogisticRegression
    
    nodelist= G.nodes()
    department= [node[1]['Department'] for node in list(G.nodes(data=True)) ]
    manage_sal= [node[1]['ManagementSalary'] for node in list(G.nodes(data=True))]
    node_df= pd.DataFrame(nodelist)
    node_df['Department']= department
    #Adding the degrees as a feature
    node_df['Degree']= [G.degree(node) for node in nodelist]
    node_df['ManagementSalary']= manage_sal
    node_df= node_df.drop(0, axis=1)
    #Creating new dataframes for the train and testing data set
    test= node_df[node_df['ManagementSalary'].isnull()]
    train= node_df[node_df['ManagementSalary'].isnull()==False]
    y_train= train['ManagementSalary']
    X_train= train[['Department','Degree']]
    X_test= test[['Department','Degree']]
    #Setting the model up
    lr= LogisticRegression()
    lr.fit(X_train, y_train)
    probabilities= lr.predict_proba(X_test)[:,1]
    final_probs= pd.Series([p for p in probabilities], index = X_test.index)
 
    return final_probs



### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [29]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head()

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [35]:
def new_connections_predictions():
    '''Perform some feature extraction and then use those are dependent variables for the prediction'''
    from sklearn.linear_model import LogisticRegression
    #Jaccard Coefficient
    edge_pairs= list(future_connections.index)
    #Extraction of the coefficients
    jc_vals=[list(nx.jaccard_coefficient(G,[edge]))[0][2] for edge in edge_pairs]
    resource= [list(nx.resource_allocation_index(G,[edge]))[0][2] for edge in edge_pairs]
    preference= [list(nx.preferential_attachment(G,[edge]))[0][2] for edge in edge_pairs]
    #Adding the new features to the dataframe
    future_connections['jaccard']= jc_vals
    future_connections['resource']= resource
    future_connections['preference']= preference
    
    #Training and testing data
    train= future_connections[future_connections['Future Connection'].isnull()==False]
    test = future_connections[future_connections['Future Connection'].isnull()]
    X_train=train[['jaccard','resource','preference']]
    y_train=train['Future Connection']
    X_test= test[['jaccard','resource','preference']]
    
    #Training the model and fitting the data
    lr= LogisticRegression()
    lr.fit(X_train,y_train)
    predictions= lr.predict_proba(X_test)[:,1]
    predicted_series= pd.Series([p for p in predictions], index= X_test.index)
    
    return predicted_series


(107, 348)    0.024692
(542, 751)    0.011674
(20, 426)     0.457538
(50, 989)     0.011524
(942, 986)    0.011366
(324, 857)    0.011545
(13, 710)     0.216688
(19, 271)     0.189313
(319, 878)    0.011473
(659, 707)    0.011659
(49, 843)     0.011427
(208, 893)    0.011622
(377, 469)    0.020393
(405, 999)    0.017469
(129, 740)    0.016286
(292, 618)    0.043843
(239, 689)    0.011463
(359, 373)    0.016070
(53, 523)     0.043756
(276, 984)    0.011435
(202, 997)    0.011404
(604, 619)    0.409571
(270, 911)    0.011465
(261, 481)    0.082378
(200, 450)    0.863053
(213, 634)    0.011721
(644, 735)    0.193474
(346, 553)    0.012108
(521, 738)    0.012904
(422, 953)    0.024256
                ...   
(672, 848)    0.011465
(28, 127)     0.744688
(202, 661)    0.012176
(54, 195)     0.999982
(295, 864)    0.011568
(814, 936)    0.011698
(839, 874)    0.011366
(139, 843)    0.011555
(461, 544)    0.013503
(68, 487)     0.013611
(622, 932)    0.011604
(504, 936)    0.023016
(479, 528) 